The Field class is the primary container for numerical simulation data. It combines a data array with metadata about the data's meaning, location, and structure.


| **Property**        | **Type**           | **Description**                                      |
|----------------------|--------------------|------------------------------------------------------|
| `data`              | `numpy.ndarray`   | Numerical data array                                |
| `scoping`           | `Scoping`         | Entity IDs this data applies to                     |
| `location`          | `str`             | Where data is located (nodal, elemental, etc.)      |
| `field_definition`  | `FieldDefinition` | Complete metadata specification                     |
| `unit`              | `str`             | Physical unit                                       |
| `dimensionality`    | `Dimensionality`  | Data shape and nature                               |


In [1]:
from ansys.dpf import core as dpf

# Create a simple field with nodal displacement data
field = dpf.Field() # You can define type of data, default vector

# Define the location (nodal data)
field.location = dpf.locations.nodal

# Create sample displacement data for 5 nodes (3D displacements: x, y, z)
import numpy as np
displacement_data = np.array([
    [0.1, 0.2, 0.3],  # Node 1
    [0.4, 0.5, 0.6],  # Node 2
    [0.7, 0.8, 0.9],  # Node 3
    [1.0, 1.1, 1.2],  # Node 4
    [1.3, 1.4, 1.5]   # Node 5
])

# Assign data to the field
field.data = displacement_data

# Define scoping (node IDs)
field.scoping.ids = [1, 2, 3, 4, 5]

# Set unit
field.unit = "m"

# Display field information
print("Field Location:", field.location)
print("Field Unit:", field.unit)
print("Field Data Shape:", field.data.shape)
print("Field Scoping IDs:", field.scoping.ids)
print("\nField Data:\n", field.data)

Field Location: Nodal
Field Unit: m
Field Data Shape: (5, 3)
Field Scoping IDs: [1 2 3 4 5]

Field Data:
 [[0.1 0.2 0.3]
 [0.4 0.5 0.6]
 [0.7 0.8 0.9]
 [1.  1.1 1.2]
 [1.3 1.4 1.5]]


You can create a `Field` from an actual model file by loading a result file and extracting displacement data:

```markdown
## Working with Model Files

To work with real simulation data, you can load a model file and extract fields directly from the results. Here's an example using a result file:

```python
# Load a model from a result file
model = dpf.Model('path/to/your/result.rst')

# Get the displacement field from the model
displacement_field = model.results.displacement()

# Extract the field for the first time step
field_from_model = displacement_field.outputs.fields_container()[0]

# Display field information
print("Field Location:", field_from_model.location)
print("Field Unit:", field_from_model.unit)
print("Number of entities:", len(field_from_model.scoping))
print("Field Data Shape:", field_from_model.data.shape)
print("\nFirst 5 displacement values:\n", field_from_model.data[:5])
```

This extracts displacement data directly from your analysis results, automatically including the correct scoping, units, and metadata.
```

In [2]:
from ansys.dpf.core import examples
model = dpf.Model(examples.download_transient_result())

# Get the displacement field from the model
displacement_field = model.results.displacement()

# Extract the field for the first time step
field_from_model = displacement_field.outputs.fields_container()[0]

# Display field information
print("Field Location:", field_from_model.location)
print("Field Unit:", field_from_model.unit)
print("Number of entities:", len(field_from_model.scoping))
print("Field Data Shape:", field_from_model.data.shape)
print("\nFirst 5 displacement values:\n", field_from_model.data[:5])

Field Location: Nodal
Field Unit: m
Number of entities: 3820
Field Data Shape: (3820, 3)

First 5 displacement values:
 [[ 6.25586668e-03 -1.39243136e-02  2.42697211e-05]
 [ 1.79675948e-02 -2.74812825e-02  1.83822050e-05]
 [-6.72664571e-03 -3.21373459e-02  1.67159110e-04]
 [-5.36085698e-03 -1.98665553e-02  1.09649059e-04]
 [ 1.21117307e-02 -2.07027981e-02  2.13259630e-05]]


## Scoping

The `Scoping` class defines which entities (nodes, elements, etc.) data applies to through lists of IDs and location information. It answers the question: "Which entities does this data belong to?"

| **Property** | **Type**        | **Description**                               |
|--------------|-----------------|-----------------------------------------------|
| `ids`        | `numpy.ndarray` | Array of entity IDs                           |
| `location`   | `str`           | Entity type (nodal, elemental, etc.)          |
| `size`       | `int`           | Number of entities                            |

**Key Methods:**
- `set_id(index, id)` - Set ID at specific index
- `id(index)` - Get ID by index
- `index(id)` - Get index by ID

In [3]:
# Create a simple scoping for nodal data
scoping = dpf.Scoping()

# Define location (nodal entities)
scoping.location = dpf.locations.nodal

# Set the entity IDs (e.g., node IDs: 10, 20, 30, 40, 50)
scoping.ids = [10, 20, 30, 40, 50]

# Display scoping information
print("Scoping Location:", scoping.location)
print("Number of entities:", scoping.size)
print("Entity IDs:", scoping.ids)
print()

# Access individual IDs by index
print("ID at index 0:", scoping.id(0))
print("ID at index 2:", scoping.id(2))
print()

# Find index of a specific ID
print("Index of ID 30:", scoping.index(30))
print("Index of ID 50:", scoping.index(50))

Scoping Location: Nodal
Number of entities: 5
Entity IDs: [10 20 30 40 50]

ID at index 0: 10
ID at index 2: 30

Index of ID 30: 2
Index of ID 50: 4


### Working with Scoping from Model

When working with actual simulation results, scoping is automatically created and associated with fields. Here's how to access and work with scoping from a model:

In [4]:
# Get scoping from a field in the model
field = displacement_field.outputs.fields_container()[0]
field_scoping = field.scoping

# Display scoping information from the field
print("Scoping from Field:")
print("  Location:", field_scoping.location)
print("  Number of entities:", field_scoping.size)
print("  First 10 entity IDs:", field_scoping.ids[:10])
print()

# You can also get scoping from the mesh
mesh = model.metadata.meshed_region
nodes_scoping = mesh.nodes.scoping

print("Scoping from Mesh Nodes:")
print("  Location:", nodes_scoping.location)
print("  Total nodes:", nodes_scoping.size)
print("  First 5 node IDs:", nodes_scoping.ids[:5])
print("  Last 5 node IDs:", nodes_scoping.ids[-5:])

Scoping from Field:
  Location: Nodal
  Number of entities: 3820
  First 10 entity IDs: [ 525  534  533  524 2757 2773 2755 2754  529  535]

Scoping from Mesh Nodes:
  Location: Nodal
  Total nodes: 3820
  First 5 node IDs: [2 3 4 5 6]
  Last 5 node IDs: [3822 3823 3824 3825 3826]


### Creating Elemental Scoping

Scoping can be used for different location types. Here's an example with elemental data:

In [5]:
# Create an elemental scoping
elemental_scoping = dpf.Scoping()
elemental_scoping.location = dpf.locations.elemental

# Define element IDs
elemental_scoping.ids = [100, 200, 300, 400, 500]

print("Elemental Scoping:")
print("  Location:", elemental_scoping.location)
print("  Number of elements:", elemental_scoping.size)
print("  Element IDs:", elemental_scoping.ids)
print()

# Create a field with elemental data (e.g., element stress values)
stress_field = dpf.Field(nature=dpf.natures.scalar)  # Von Mises Stress
stress_field.location = dpf.locations.elemental
stress_field.scoping = elemental_scoping

# Assign stress data (scalar value per element)
stress_data = np.array([150.5, 200.3, 175.8, 190.2, 165.1])
stress_field.data = stress_data
stress_field.unit = "MPa"

print("Elemental Stress Field:")
print("  Location:", stress_field.location)
print("  Unit:", stress_field.unit)
print("  Scoping IDs:", stress_field.scoping.ids)
print("  Stress values:", stress_field.data)

Elemental Scoping:
  Location: Elemental
  Number of elements: 5
  Element IDs: [100 200 300 400 500]

Elemental Stress Field:
  Location: Elemental
  Unit: MPa
  Scoping IDs: [100 200 300 400 500]
  Stress values: [150.5 200.3 175.8 190.2 165.1]


## FieldsContainer

`FieldsContainer` is a collection of `Field` objects organized by labels such as time steps, frequency, or solution components. It allows you to manage multiple fields systematically.

**Key Features:**
- **Label Management:** Define dimensions like "time", "complex", "body"
- **add_field(label_space, field):** Add field with specific labels
- **get_field(label_space):** Retrieve field by labels
- **get_available_ids_for_label(label):** Get all IDs for a label

**Common Label Patterns:**
- Time-based: `{"time": 1}` for time step 1
- Complex: `{"time": 1, "complex": 0}` for real part at time step 1
- Body-based: `{"body": 1, "time": 5}` for body 1 at time step 5

### Creating a Simple FieldsContainer

In [6]:
# Create a fields container for time-varying displacement data

fields_container = dpf.FieldsContainer()

# Define label (time)
fields_container.labels = ["time"]

# Create fields for different time steps
for time_step in range(1, 4):
    # Create a field for this time step
    field = dpf.Field()
    field.location = dpf.locations.nodal
    field.scoping.ids = [1, 2, 3, 4, 5]
    
    # Create displacement data that varies with time
    displacement = np.array([
        [0.1 * time_step, 0.2 * time_step, 0.3 * time_step],
        [0.4 * time_step, 0.5 * time_step, 0.6 * time_step],
        [0.7 * time_step, 0.8 * time_step, 0.9 * time_step],
        [1.0 * time_step, 1.1 * time_step, 1.2 * time_step],
        [1.3 * time_step, 1.4 * time_step, 1.5 * time_step]
    ])
    field.data = displacement
    field.unit = "mm"
    
    # Add field to container with time label
    fields_container.add_field({"time": time_step}, field)

# Display container information
print("Fields Container Information:")
print("  Labels:", fields_container.labels)
print("  Number of fields:", len(fields_container))
print("  Available time steps:", fields_container.get_available_ids_for_label("time"))
print()

# Access specific fields by time step
for time_step in [1, 2, 3]:
    field = fields_container.get_field({"time": time_step})
    print(f"Time Step {time_step}:")
    print(f"  First node displacement: {field.data[0]}")
    print()

Fields Container Information:
  Labels: ['time']
  Number of fields: 3
  Available time steps: [1, 2, 3]

Time Step 1:
  First node displacement: [0.1 0.2 0.3]

Time Step 2:
  First node displacement: [0.2 0.4 0.6]

Time Step 3:
  First node displacement: [0.3 0.6 0.9]



### Working with FieldsContainer from Model

When you request results from a model, DPF often returns a `FieldsContainer` containing fields for all time steps or frequencies:

In [7]:
# Get all displacement fields (all time steps)
displacement_field = model.results.displacement.on_all_time_freqs()
all_displacements = displacement_field.outputs.fields_container()

# Display container information
print("Displacement Fields Container:")
print("  Labels:", all_displacements.labels)
print("  Number of fields (time steps):", len(all_displacements))
print("  Available time steps:", all_displacements.get_available_ids_for_label("time"))
print()

# Access fields by time step index
print("Accessing fields by index:")
field_0 = all_displacements[0]  # First time step
field_1 = all_displacements[1]  # Second time step
print(f"  Field 0 - Max displacement: {np.max(np.linalg.norm(field_0.data, axis=1)):.6f} {field_0.unit}")
print(f"  Field 1 - Max displacement: {np.max(np.linalg.norm(field_1.data, axis=1)):.6f} {field_1.unit}")
print()

# Access fields by label
print("Accessing fields by time label:")
time_ids = all_displacements.get_available_ids_for_label("time")
for i, time_id in enumerate(time_ids[:3]):  # Show first 3 time steps
    field = all_displacements.get_field({"time": time_id})
    max_disp = np.max(np.linalg.norm(field.data, axis=1))
    print(f"  Time ID {time_id}: Max displacement = {max_disp:.6f} {field.unit}")

Displacement Fields Container:
  Labels: ['time']
  Number of fields (time steps): 35
  Available time steps: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]

Accessing fields by index:
  Field 0 - Max displacement: 0.000000 m
  Field 1 - Max displacement: 0.000627 m

Accessing fields by time label:
  Time ID 1: Max displacement = 0.000000 m
  Time ID 2: Max displacement = 0.000627 m
  Time ID 3: Max displacement = 0.002509 m


### Multi-Label FieldsContainer

You can create containers with multiple labels (e.g., time and complex for harmonic analysis):

In [8]:
# Create a multi-label container (time and complex components)
multi_label_container = dpf.FieldsContainer()
multi_label_container.labels = ["time", "complex"]

# Add fields for different time steps and complex components
# complex: 0 = real part, 1 = imaginary part
for time_step in [1, 2]:
    for complex_part in [0, 1]:
        field = dpf.Field()
        field.location = dpf.locations.nodal
        field.scoping.ids = [1, 2, 3]
        
        # Create different data for real and imaginary parts
        if complex_part == 0:  # Real part
            data = np.array([
                [1.0 * time_step, 2.0 * time_step, 3.0 * time_step],
                [4.0 * time_step, 5.0 * time_step, 6.0 * time_step],
                [7.0 * time_step, 8.0 * time_step, 9.0 * time_step]
            ])
        else:  # Imaginary part
            data = np.array([
                [0.5 * time_step, 1.0 * time_step, 1.5 * time_step],
                [2.0 * time_step, 2.5 * time_step, 3.0 * time_step],
                [3.5 * time_step, 4.0 * time_step, 4.5 * time_step]
            ])
        
        field.data = data
        field.unit = "mm"
        
        # Add field with both labels
        multi_label_container.add_field({"time": time_step, "complex": complex_part}, field)

print("Multi-Label Container:")
print(multi_label_container)
print("  Labels:", multi_label_container.labels)
print("  Number of fields:", len(multi_label_container))
print("  Available time IDs:", multi_label_container.get_available_ids_for_label("time"))
print("  Available complex IDs:", multi_label_container.get_available_ids_for_label("complex"))
print()

# Access specific field by both labels
field_real_t1 = multi_label_container.get_field({"time": 1, "complex": 0})
field_imag_t1 = multi_label_container.get_field({"time": 1, "complex": 1})
field_real_t2 = multi_label_container.get_field({"time": 2, "complex": 0})

print("Field at Time=1, Complex=0 (Real):")
print(f"  First node data: {field_real_t1.data[0]}")
print()
print("Field at Time=1, Complex=1 (Imaginary):")
print(f"  First node data: {field_imag_t1.data[0]}")
print()
print("Field at Time=2, Complex=0 (Real):")
print(f"  First node data: {field_real_t2.data[0]}")

Multi-Label Container:
DPF  Fields Container
  with 4 field(s)
  defined on labels: complex time 

  with:
  - field 0 {complex:  0, time:  1} with Nodal location, 3 components and 3 entities.
  - field 1 {complex:  1, time:  1} with Nodal location, 3 components and 3 entities.
  - field 2 {complex:  0, time:  2} with Nodal location, 3 components and 3 entities.
  - field 3 {complex:  1, time:  2} with Nodal location, 3 components and 3 entities.

  Labels: ['complex', 'time']
  Number of fields: 4
  Available time IDs: [1, 2]
  Available complex IDs: [0, 1]

Field at Time=1, Complex=0 (Real):
  First node data: [1. 2. 3.]

Field at Time=1, Complex=1 (Imaginary):
  First node data: [0.5 1.  1.5]

Field at Time=2, Complex=0 (Real):
  First node data: [2. 4. 6.]


### Iterating Through FieldsContainer

You can iterate through all fields in a container:

In [9]:
# Iterate through the first few fields in the displacement container
print("Iterating through displacement fields:")
print()

for i, field in enumerate(all_displacements):
    if i >= 3:  # Show only first 3 fields
        break
    
    # Calculate maximum displacement magnitude
    displacement_magnitudes = np.linalg.norm(field.data, axis=1)
    max_disp = np.max(displacement_magnitudes)
    avg_disp = np.mean(displacement_magnitudes)
    
    print(f"Field {i}:")
    print(f"  Number of nodes: {len(field.scoping)}")
    print(f"  Max displacement: {max_disp:.6f} {field.unit}")
    print(f"  Avg displacement: {avg_disp:.6f} {field.unit}")
    print()

# You can also get fields by label and iterate
print("Fields at specific time steps:")
time_steps = all_displacements.get_available_ids_for_label("time")
for time_id in time_steps[:3]:
    field = all_displacements.get_field({"time": time_id})
    max_disp = np.max(np.linalg.norm(field.data, axis=1))
    print(f"  Time {time_id}: Max displacement = {max_disp:.6f} {field.unit}")

Iterating through displacement fields:

Field 0:
  Number of nodes: 3820
  Max displacement: 0.000000 m
  Avg displacement: 0.000000 m

Field 1:
  Number of nodes: 3820
  Max displacement: 0.000627 m
  Avg displacement: 0.000455 m

Field 2:
  Number of nodes: 3820
  Max displacement: 0.002509 m
  Avg displacement: 0.001823 m

Fields at specific time steps:
  Time 1: Max displacement = 0.000000 m
  Time 2: Max displacement = 0.000627 m
  Time 3: Max displacement = 0.002509 m


## Practice Exercises

Test your understanding of Fields, Scoping, and FieldsContainer with these exercises!

### Exercise 1: Create a Temperature Field
Create a field representing temperature data for 7 nodes with the following requirements:
- Location: nodal
- Node IDs: [5, 10, 15, 20, 25, 30, 35]
- Temperature values: [25.5, 30.2, 28.7, 32.1, 29.5, 31.0, 27.8] (in Celsius)
- Unit: "°C"

Print the field information and find the maximum temperature and its node ID.

In [10]:
# Your code for Exercise 1 here

### Exercise 2: Working with Scoping
Create two scoping objects:
1. **Scoping A**: Nodal location with IDs [100, 200, 300, 400, 500]
2. **Scoping B**: Elemental location with IDs [10, 20, 30, 40]

For Scoping A:
- Find the index of node ID 300
- Get the node ID at index 4
- Print the total size

In [11]:
# Your code for Exercise 2 here

### Exercise 3: Building a FieldsContainer
Create a FieldsContainer with velocity data for 3 time steps:
- Use label "time"
- For each time step (1, 2, 3), create a field with:
  - 4 nodes (IDs: [1, 2, 3, 4])
  - 3D velocity vectors that increase with time (multiply base velocity by time step)
  - Base velocities: [[1.0, 0.0, 0.5], [0.8, 0.2, 0.6], [1.2, 0.1, 0.4], [0.9, 0.3, 0.7]]
  - Unit: "m/s"

Then:
- Print the number of fields in the container
- Access and print the velocity at time step 2 for node 3
- Calculate the average velocity magnitude across all time steps

In [12]:
# Your code for Exercise 3 here

---

## Solutions (Try the exercises first!)

### Solution 1: Create a Temperature Field

In [15]:
# Solution 1: Create a Temperature Field
temp_field = dpf.Field(nature=dpf.natures.scalar)
temp_field.location = dpf.locations.nodal

# Set node IDs
temp_field.scoping.ids = [5, 10, 15, 20, 25, 30, 35]

# Set temperature data (scalar values)
temperature_data = np.array([25.5, 30.2, 28.7, 32.1, 29.5, 31.0, 27.8])
temp_field.data = temperature_data

# Set unit
temp_field.unit = "C"

# Print field information
print("Temperature Field Information:")
print(f"  Location: {temp_field.location}")
print(f"  Unit: {temp_field.unit}")
print(f"  Number of nodes: {len(temp_field.scoping)}")
print(f"  Node IDs: {temp_field.scoping.ids}")
print()

# Find maximum temperature
max_temp_idx = np.argmax(temp_field.data)
max_temp = temp_field.data[max_temp_idx]
max_temp_node = temp_field.scoping.ids[max_temp_idx]

print(f"  Maximum temperature: {max_temp} {temp_field.unit}")
print(f"  Located at node: {max_temp_node}")

Temperature Field Information:
  Location: Nodal
  Unit: C
  Number of nodes: 7
  Node IDs: [ 5 10 15 20 25 30 35]

  Maximum temperature: 32.1 C
  Located at node: 20


### Solution 2: Working with Scoping

In [16]:
# Solution 2: Working with Scoping

# Create Scoping A - Nodal
scoping_a = dpf.Scoping()
scoping_a.location = dpf.locations.nodal
scoping_a.ids = [100, 200, 300, 400, 500]

# Create Scoping B - Elemental
scoping_b = dpf.Scoping()
scoping_b.location = dpf.locations.elemental
scoping_b.ids = [10, 20, 30, 40]

print("Scoping A (Nodal):")
print(f"  Location: {scoping_a.location}")
print(f"  IDs: {scoping_a.ids}")
print(f"  Size: {scoping_a.size}")
print()

# Find index of node ID 300
idx_of_300 = scoping_a.index(300)
print(f"  Index of node ID 300: {idx_of_300}")

# Get node ID at index 4
id_at_idx_4 = scoping_a.id(4)
print(f"  Node ID at index 4: {id_at_idx_4}")
print()

print("Scoping B (Elemental):")
print(f"  Location: {scoping_b.location}")
print(f"  IDs: {scoping_b.ids}")
print(f"  Size: {scoping_b.size}")

Scoping A (Nodal):
  Location: Nodal
  IDs: [100 200 300 400 500]
  Size: 5

  Index of node ID 300: 2
  Node ID at index 4: 500

Scoping B (Elemental):
  Location: Elemental
  IDs: [10 20 30 40]
  Size: 4


### Solution 3: Building a FieldsContainer

In [17]:
# Solution 3: Building a FieldsContainer

# Create the container
velocity_container = dpf.FieldsContainer()
velocity_container.labels = ["time"]

# Base velocity data
base_velocity = np.array([
    [1.0, 0.0, 0.5],
    [0.8, 0.2, 0.6],
    [1.2, 0.1, 0.4],
    [0.9, 0.3, 0.7]
])

# Create fields for time steps 1, 2, 3
for time_step in range(1, 4):
    field = dpf.Field()
    field.location = dpf.locations.nodal
    field.scoping.ids = [1, 2, 3, 4]
    
    # Multiply base velocity by time step
    field.data = base_velocity * time_step
    field.unit = "m/s"
    
    # Add to container
    velocity_container.add_field({"time": time_step}, field)

# Print container information
print("Velocity FieldsContainer:")
print(f"  Labels: {velocity_container.labels}")
print(f"  Number of fields: {len(velocity_container)}")
print()

# Access velocity at time step 2 for node 3
field_t2 = velocity_container.get_field({"time": 2})
# Node 3 is at index 2 (0-based indexing)
velocity_node3_t2 = field_t2.data[2]
print(f"  Velocity at time step 2, node 3: {velocity_node3_t2} {field_t2.unit}")
print()

# Calculate average velocity magnitude across all time steps
print("  Velocity magnitudes per time step:")
all_magnitudes = []
for time_step in range(1, 4):
    field = velocity_container.get_field({"time": time_step})
    magnitudes = np.linalg.norm(field.data, axis=1)
    avg_mag = np.mean(magnitudes)
    all_magnitudes.append(avg_mag)
    print(f"    Time step {time_step}: Avg magnitude = {avg_mag:.4f} {field.unit}")

overall_avg = np.mean(all_magnitudes)
print()
print(f"  Overall average velocity magnitude: {overall_avg:.4f} m/s")

Velocity FieldsContainer:
  Labels: ['time']
  Number of fields: 3

  Velocity at time step 2, node 3: [2.4 0.2 0.8] m/s

  Velocity magnitudes per time step:
    Time step 1: Avg magnitude = 1.1464 m/s
    Time step 2: Avg magnitude = 2.2928 m/s
    Time step 3: Avg magnitude = 3.4393 m/s

  Overall average velocity magnitude: 2.2928 m/s
